In [13]:
# Import the necessary libraries
import pandas as pd
import requests
from datetime import datetime

In [14]:
# Read the CSV file into a DataFrame
df = pd.read_csv('city_country.csv', delimiter=';')

# Print the columns in the DataFrame
print(df.columns)

# You can change the country here
country_you_want = 'Sweden'

# Filter data for cities in Sweden
df_country = df[df['country'] == country_you_want]

# Print the rows in the DataFrame, after applying the filter

print(df_country)

Index(['city_id', 'city_acsii', 'country'], dtype='object')
       city_id        city_acsii country
1            2             `Adra   Syria
2            3            `Afrin   Syria
12          13            `Amuda   Syria
13          14           `Anadan   Syria
16          17     `Ayn al `Arab   Syria
...        ...               ...     ...
39155    39156            Tartus   Syria
39172    39173             Tasil   Syria
39262    39263  Tayyibat al Imam   Syria
40810    40811          Turmanin   Syria
43687    43688            Yabrud   Syria

[135 rows x 3 columns]


In [15]:
# Provide the API Key to call Rest API
# By signing up on this page: https://www.weatherapi.com/
api_key = "489f8ed5010e4d1082a230213230711"

# Create lists to store data for each column
all_data = []

# Create lists for error data
error_data = []

# Iterate through each row in the 'city_acsii' column
for city in df_country['city_acsii']:
    # Construct the API link for each city
    api_link = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={city}"

    # Make an API request
    response = requests.get(api_link)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Access and use the response data (json() method returns a dictionary)
        data = response.json()

        # Extract relevant information from the JSON response
        location = data.get('location', {})
        current = data.get('current', {})
        condition = current.get('condition', {})

        # Append the extracted data to the list
        all_data.append([
            # location
            location.get('name'),
            location.get('region'),
            location.get('country'),
            location.get('lat'),
            location.get('lon'),
            location.get('tz_id'),
            location.get('localtime_epoch'),
            location.get('localtime'),
            # current
            current.get('last_updated_epoch'),
            current.get('last_updated'),
            current.get('temp_c'),
            current.get('temp_f'),
            current.get('is_day'),
            # condition
            condition.get('text'),
            condition.get('icon'),
            # current again
            current.get('wind_mph'),
            current.get('wind_kph'),
            current.get('wind_degree'),
            current.get('wind_dir'),
            current.get('pressure_mb'),
            current.get('pressure_in'),
            current.get('precip_mm'),
            current.get('precip_in'),
            current.get('humidity'),
            current.get('cloud'),
            current.get('feelslike_c'),
            current.get('feelslike_f'),
            current.get('vis_km'),
            current.get('vis_miles'),
            current.get('uv'),
            current.get('gust_mph'),
            current.get('gust_kph')
            # Add more fields as needed
        ])
    else:
        # Add error message as dictionary if the request was not successful
        error_data.append({'City': city, 'StatusCode': response.status_code})



In [16]:
# Create a DataFrame from the list of lists
result_df = pd.DataFrame(
    all_data,
    columns=[
        # location
        'City', 'Region', 'Country', 'Latitude', 'Longitude', 'TimeZone', 'LocalTimeEpoch', 'LocalTime',
        # current
        'LastUpdatedEpoch', 'LastUpdated', 'Temperature_C', 'Temperature_F', 'IsDay',
        # condition
        'ConditionText', 'ConditionIcon', 
        # current again
        'WindMPH', 'WindKPH', 'WindDegree', 'WindDir',
        'PressureMB', 'PressureIn', 'PrecipMM', 'PrecipIn', 'Humidity', 'Cloud', 'FeelsLike_C',
        'FeelsLike_F', 'VisibilityKM', 'VisibilityMiles', 'UV', 'GustMPH', 'GustKPH'
    ]
)

# Create a DataFrame from the previous error message dictionary
error_result_df = pd.DataFrame(error_data, columns= ['City', 'StatusCode'])

In [17]:
# Create a DataFrame from the list of error data
error_df = pd.DataFrame(error_data)

# Generate a timestamp for the output file names
timestamp = datetime.now().strftime("%Y%m%d%H%M")

# Save the DataFrame to a CSV file
result_df.to_csv(f'output_data_{timestamp}_{country_you_want}.csv', index=False)
error_df.to_csv(f'output_error_data__{timestamp}_{country_you_want}.csv', index=False)